## Import sqlite packages & db; create function to pass query to

In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
   
    
def select_all_tasks(conn, query):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
   
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

        
        
query1 = """
SELECT *
FROM FACILITIES
"""

p2 = "2. Query all tasks"

def ko_sql(query, problem = ""):
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print(problem)
        select_all_tasks(conn, query)
 

ko_sql(query1)

2.6.0

(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


## Answer questions by writing script and passing to function.

In [2]:
Q10= 'Q10: Produce a list of facilities with a total revenue less than 1000.'

query10 = """
SELECT
    Facility_Name
    ,Annual_Rev
FROM ( SELECT 
       Facility_Name
        ,sum(rev) AS Annual_Rev
    FROM  (SELECT
            b.memid
            ,b.facid
            ,f.name as Facility_Name
            ,(CASE WHEN b.memid = 0 THEN f.guestcost
                ELSE f.membercost END) * b.slots
                AS rev
        FROM bookings b
        LEFT JOIN facilities f
        ON b.facid = f.facid
        ) AS all_b
    GROUP BY Facility_Name) AS Annual_Frev
WHERE
    Annual_Rev < 1000
GROUP BY Facility_Name
ORDER BY Annual_Rev, Facility_Name
"""

ko_sql(query10, Q10)

2.6.0
Q10: Produce a list of facilities with a total revenue less than 1000.
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


In [3]:
Q11=  'Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order'

query11 = """
SELECT
    (m.surname || ", " || m.firstname) AS mem_last_name_first
    ,(rec.surname || ", " || rec.firstname) AS rec_last_name_first 
FROM Members m
LEFT JOIN ( 
    SELECT 
       memid
       ,surname
       ,firstname
    FROM Members
        ) AS rec
ON m.recommendedby = rec.memid
WHERE m.memid <> 0
AND m.recommendedby >0
ORDER BY mem_last_name_first;
"""

ko_sql(query11, Q11)

2.6.0
Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Jack', 'Smith, Darren')
('Stibbons, Ponder', 'Tracy, Burton')
('Worthington-Smyth, Henry', 'Smith, Tracy')


In [4]:
#Unclear if this is asking for a count of usage, not including guest--or a list of usage by member. So I did both

Q12 = 'Find the facilities with their usage by member, but not guests'

query12a = """
SELECT 
    f.name AS Facility_Name
    ,COUNT(b.bookid) AS Member_Usage
FROm bookings b
LEFT JOIN facilities f
ON b.facid = f.facid
GROUP BY 1
ORDER BY 1;

"""

ko_sql(query12a, Q12)

2.6.0
Find the facilities with their usage by member, but not guests
('Badminton Court', 383)
('Massage Room 1', 629)
('Massage Room 2', 111)
('Pool Table', 836)
('Snooker Table', 444)
('Squash Court', 440)
('Table Tennis', 403)
('Tennis Court 1', 408)
('Tennis Court 2', 389)


In [5]:

Q12 = 'Find the facilities with their usage by member, but not guests'

query12b = """
SELECT 
   (m.surname || ", " || m.firstname) AS Member_Name
   ,all_b.Facility_Name
   ,COUNT(all_b.bookid) AS Member_Usage 
FROM (
    SELECT 
        b.bookid
        ,b.memid
        ,b.facid
        ,f.name AS Facility_Name
    FROM Bookings b
    LEFT JOIN Facilities f
    ON b.facid = f.facid
    WHERE b.memid <> 0
    ) as all_b
LEFT JOIN members m
ON all_b.memid = m.memid
GROUP BY 1,2 
ORDER BY 1, 2


"""

ko_sql(query12b, Q12)

2.6.0
Find the facilities with their usage by member, but not guests
('Bader, Florence', 'Badminton Court', 9)
('Bader, Florence', 'Massage Room 2', 2)
('Bader, Florence', 'Pool Table', 23)
('Bader, Florence', 'Snooker Table', 33)
('Bader, Florence', 'Squash Court', 2)
('Bader, Florence', 'Table Tennis', 42)
('Bader, Florence', 'Tennis Court 1', 1)
('Bader, Florence', 'Tennis Court 2', 8)
('Baker, Anne', 'Badminton Court', 10)
('Baker, Anne', 'Massage Room 1', 3)
('Baker, Anne', 'Massage Room 2', 2)
('Baker, Anne', 'Pool Table', 12)
('Baker, Anne', 'Squash Court', 49)
('Baker, Anne', 'Table Tennis', 1)
('Baker, Anne', 'Tennis Court 1', 6)
('Baker, Anne', 'Tennis Court 2', 35)
('Baker, Timothy', 'Badminton Court', 7)
('Baker, Timothy', 'Massage Room 1', 24)
('Baker, Timothy', 'Pool Table', 85)
('Baker, Timothy', 'Squash Court', 5)
('Baker, Timothy', 'Table Tennis', 24)
('Baker, Timothy', 'Tennis Court 1', 14)
('Baker, Timothy', 'Tennis Court 2', 7)
('Boothe, Tim', 'Badminton Court', 12)

In [6]:
Q13 = "Q13: Find the facilities usage by month, but not guests"

query13 = """
SELECT
    Booking_Month
    ,Facility_Name
    ,COUNT(monthly_b.bookid)
FROM ( 
    SELECT
        b.bookid
        ,f.name AS Facility_Name
        ,SUBSTR(b.starttime, 1, 7) AS Booking_Month
    FROM Bookings b
    JOIN Facilities f
    ON b.facid = f.facid
    WHERE b.memid <> 0
    ) AS monthly_b
GROUP BY 1, 2
ORDER BY 1, 2;
"""



ko_sql(query13, Q13)

2.6.0
Q13: Find the facilities usage by month, but not guests
('2012-07', 'Badminton Court', 51)
('2012-07', 'Massage Room 1', 77)
('2012-07', 'Massage Room 2', 4)
('2012-07', 'Pool Table', 103)
('2012-07', 'Snooker Table', 68)
('2012-07', 'Squash Court', 23)
('2012-07', 'Table Tennis', 48)
('2012-07', 'Tennis Court 1', 65)
('2012-07', 'Tennis Court 2', 41)
('2012-08', 'Badminton Court', 132)
('2012-08', 'Massage Room 1', 153)
('2012-08', 'Massage Room 2', 9)
('2012-08', 'Pool Table', 272)
('2012-08', 'Snooker Table', 154)
('2012-08', 'Squash Court', 85)
('2012-08', 'Table Tennis', 143)
('2012-08', 'Tennis Court 1', 111)
('2012-08', 'Tennis Court 2', 109)
('2012-09', 'Badminton Court', 161)
('2012-09', 'Massage Room 1', 191)
('2012-09', 'Massage Room 2', 14)
('2012-09', 'Pool Table', 408)
('2012-09', 'Snooker Table', 199)
('2012-09', 'Squash Court', 87)
('2012-09', 'Table Tennis', 194)
('2012-09', 'Tennis Court 1', 132)
('2012-09', 'Tennis Court 2', 126)
